In [4]:
import os

In [7]:
# List with the databases we're including in the preprocessing
databases = ['vfdb/']
# Path to original data directory
databases_dir = "data_original/"

In [ ]:
for database in databases:
    database_path = os.path.join(databases_dir, database)

    # LF a `RECORDS` file - contains the names of all the records in that database
    records_file = os.path.join(database_path, "RECORDS")

    with open(records_file, "r") as file:
        record_names = file.read().splitlines()

    for record_name in record_names:
        record_path = os.path.join(database_path, record_name)

        # TODO: preprocessing with every file or whatever
        print(record_path)
